# Benchmark m4

How to fit several model and evaluate its predictions on three of subsets of M4 data. 


#### functools.partial

Return a new `partial` object which when called will behave like *func* called with the positional arguments args and keyword arguments keywords. 

- [Source](https://docs.python.org/2/library/functools.html)

## Methods  

Brief overview of the used methods

### `DeepAREstimator`

- [`gluonts.model.deepar`](https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html)

Construct a DeepAR estimator, implements an RNN-based model, close to the one described in

- Salinas, David, Valentin Flunkert, and Jan Gasthaus. “DeepAR: Probabilistic forecasting with autoregressive recurrent networks.” arXiv preprint arXiv:1704.04110 (2017).


### `MQCNNEstimator`

- [`gluonts.model.seq2seq`](http://gluon-ts.mxnet.io/api/gluonts/gluonts.model.seq2seq.html)

An `MQDNNEstimator` with Convolutional Neural Network (CNN) as an encoder. Implements the MQ-CNN Forecaster, proposed in 

- Wen, Ruofeng, et al. “A multi-horizon quantile recurrent forecaster.” arXiv preprint arXiv:1711.11053 (2017).


### `SimpleFeedForwardEstimator`

- [`gluonts.model.simple_feedforward`](http://gluon-ts.mxnet.io/api/gluonts/gluonts.model.simple_feedforward.html)

A simple multilayer perceptron model predicting the next target time-steps given the previous ones. 



In [1]:
# imports
import pandas as pd

from functools import partial

# gluon imports
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.distribution.piecewise_linear import PiecewiseLinearOutput
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions

from gluonts.model.deepar import DeepAREstimator
from gluonts.model.seq2seq import MQCNNEstimator
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator

from gluonts.trainer import Trainer

import pprint


INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU


Number of series in m4 data: 

- Yearly: 23,000
- Quarterly: 24,000
- Monthly: 48,000
- Weekly: 359
- Daily: 4227
- Hourly: 414



In [2]:
datasets = [
    "m4_hourly",
    "m4_daily",
    "m4_weekly"
]

print(datasets)

epochs = 100
num_batches_per_epoch = 50


['m4_hourly', 'm4_daily', 'm4_weekly']


In [3]:
estimators = [
    partial(
        SimpleFeedForwardEstimator,
        trainer=Trainer(
            epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
        ),
    ),
    partial(
        DeepAREstimator,
        trainer=Trainer(
            epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
        ),
    ),
    partial(
        MQCNNEstimator,
        trainer=Trainer(
            epochs=epochs, num_batches_per_epoch=num_batches_per_epoch
        ),
    ),
]


INFO:root:Using CPU
INFO:root:Using CPU
INFO:root:Using CPU


In [4]:
# evaluation
def evaluate(dataset_name, estimator):
    dataset=get_dataset(dataset_name)
    estimator = estimator(
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq
    )
    
    print("evaluating {} on {}".format(estimator, dataset))
    
    predictor=estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    
    )
    
    pprint.pprint(agg_metrics)
    
    eval_dict = agg_metrics
    eval_dict["dataset"] = dataset_name
    eval_dict["estimator"] = type(estimator).__name__
    return eval_dict
    

In [5]:
%%time

if __name__=="__main__":
    results=[]
    
    for dataset_name in datasets:
        for estimator in estimators:
            
            try:
                results.append(evaluate(dataset_name, estimator))
            except Exception as e:
                print(str(e))
    
    df=pd.DataFrame(results)
    
    sub_df=df[
        [
            "dataset",
            "estimator",
            "RMSE",
            "mean_wQuantileLoss",
            "MASE",
            "sMAPE",
            "MSIS",
        ]
    ]
    
    print(sub_df.to_string())

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_hourly.


evaluating gluonts.model.simple_feedforward._estimator.SimpleFeedForwardEstimator(batch_normalization=False, context_length=None, distr_output=gluonts.distribution.student_t.StudentTOutput(), freq="H", mean_scaling=True, num_hidden_dimensions=[40, 40], num_parallel_samples=100, prediction_length=48, trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=None, epochs=100, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=50, patience=10, weight_decay=1e-08)) on TrainDatasets(metadata=<MetaData freq='H' target=None feat_static_cat=[<CategoricalFeatureInfo name='feat_static_cat' cardinality='414'>] feat_static_real=[] feat_dynamic_real=[] feat_dynamic_cat=[] prediction_length=48>, train=<gluonts.dataset.common.FileDataset object at 0x0000025230DA4898>, test=<gluonts.dataset.common.FileDataset object at 0x0000025230DB06D8>)


INFO:root:Start model training
INFO:root:Number of parameters in SimpleFeedForwardTrainingNetwork: 1963
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 56.97it/s, avg_epoch_loss=5.43]
INFO:root:Epoch[0] Elapsed time 0.889 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=5.430500
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:01<00:00, 44.16it/s, avg_epoch_loss=4.41]
INFO:root:Epoch[1] Elapsed time 1.140 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=4.411243
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 64.89it/s, avg_epoch_loss=4.54]
INFO:root:Epoch[2] Elapsed time 0.781 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=4.542153
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:00<00:00, 66.00it/s, avg_epoch_loss=4.59]
INFO:root:Epoch[3] Elapsed time 0.778 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=4.592535
INFO:root:Epoch[4] Learning rate

INFO:root:Epoch[73] Learning rate is 6.25e-05
100%|██████████| 50/50 [00:00<00:00, 74.78it/s, avg_epoch_loss=4.02]
INFO:root:Epoch[73] Elapsed time 0.676 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=4.017392
INFO:root:Epoch[74] Learning rate is 6.25e-05
100%|██████████| 50/50 [00:00<00:00, 71.73it/s, avg_epoch_loss=4.04]
INFO:root:Epoch[74] Elapsed time 0.708 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=4.036949
INFO:root:Epoch[75] Learning rate is 6.25e-05
100%|██████████| 50/50 [00:00<00:00, 67.56it/s, avg_epoch_loss=4.12]
INFO:root:Epoch[75] Elapsed time 0.751 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=4.124209
INFO:root:Epoch[76] Learning rate is 6.25e-05
100%|██████████| 50/50 [00:00<00:00, 77.93it/s, avg_epoch_loss=4.15]
INFO:root:Epoch[76] Elapsed time 0.648 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=4.148879
INFO:root:Epoch[77] Learning rate is 6.25e-05
100%|██████████| 50/50 [00:00<00:00, 59.42it/s, avg_epoch_loss=4.0

{'Coverage[0.1]': 0.12806964573268922,
 'Coverage[0.2]': 0.21703904991948486,
 'Coverage[0.3]': 0.30510265700483086,
 'Coverage[0.4]': 0.4099235104669888,
 'Coverage[0.5]': 0.529589371980676,
 'Coverage[0.6]': 0.6340579710144931,
 'Coverage[0.7]': 0.7399859098228667,
 'Coverage[0.8]': 0.8231179549114339,
 'Coverage[0.9]': 0.8856682769726248,
 'MAE_Coverage': 0.022357532653426524,
 'MASE': 1.5672358709720084,
 'MSE': 5448582.082203241,
 'MSIS': 17.803665128812487,
 'ND': 0.0558745095676312,
 'NRMSE': 0.3186725613981231,
 'QuantileLoss[0.1]': 4234349.405347252,
 'QuantileLoss[0.2]': 6688490.259087372,
 'QuantileLoss[0.3]': 7854520.564018535,
 'QuantileLoss[0.4]': 8229983.418843079,
 'QuantileLoss[0.5]': 8133030.163612843,
 'QuantileLoss[0.6]': 7552375.485861397,
 'QuantileLoss[0.7]': 6705858.636860467,
 'QuantileLoss[0.8]': 5437379.945453261,
 'QuantileLoss[0.9]': 3578518.900980377,
 'RMSE': 2334.2198016046477,
 'abs_error': 8133030.116849899,
 'abs_target_mean': 7324.822041043147,
 'abs

...

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_weekly.


evaluating gluonts.model.seq2seq._mq_dnn_estimator.MQCNNEstimator(context_length=None, freq="W", mlp_final_dim=20, mlp_hidden_dimension_seq=[], prediction_length=13, quantiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], trainer=gluonts.trainer._base.Trainer(batch_size=32, clip_gradient=10.0, ctx=None, epochs=100, hybridize=True, init="xavier", learning_rate=0.001, learning_rate_decay_factor=0.5, minimum_learning_rate=5e-05, num_batches_per_epoch=50, patience=10, weight_decay=1e-08)) on TrainDatasets(metadata=<MetaData freq='W' target=None feat_static_cat=[<CategoricalFeatureInfo name='feat_static_cat' cardinality='359'>] feat_static_real=[] feat_dynamic_real=[] feat_dynamic_cat=[] prediction_length=13>, train=<gluonts.dataset.common.FileDataset object at 0x00000252348A1A58>, test=<gluonts.dataset.common.FileDataset object at 0x000002523B4A4B38>)


INFO:root:Start model training
INFO:root:Number of parameters in ForkingSeq2SeqTrainingNetwork: 359
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 12.04it/s, avg_epoch_loss=487]
INFO:root:Epoch[0] Elapsed time 4.162 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=487.450366
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:03<00:00, 13.84it/s, avg_epoch_loss=160]
INFO:root:Epoch[1] Elapsed time 3.620 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=159.818149
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:04<00:00, 10.13it/s, avg_epoch_loss=58.5]
INFO:root:Epoch[2] Elapsed time 4.944 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=58.522963
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:06<00:00,  8.15it/s, avg_epoch_loss=35.3]
INFO:root:Epoch[3] Elapsed time 6.140 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=35.311757
INFO:root:Epoch[4] Learning rate

INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=30.218393
INFO:root:Epoch[74] Learning rate is 0.0005
100%|██████████| 50/50 [00:03<00:00, 13.10it/s, avg_epoch_loss=30.3]
INFO:root:Epoch[74] Elapsed time 3.823 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=30.315948
INFO:root:Epoch[75] Learning rate is 0.0005
100%|██████████| 50/50 [00:03<00:00, 13.04it/s, avg_epoch_loss=29]
INFO:root:Epoch[75] Elapsed time 3.842 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=28.996918
INFO:root:Epoch[76] Learning rate is 0.0005
100%|██████████| 50/50 [00:03<00:00, 14.53it/s, avg_epoch_loss=31.2]
INFO:root:Epoch[76] Elapsed time 3.446 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=31.157412
INFO:root:Epoch[77] Learning rate is 0.0005
100%|██████████| 50/50 [00:03<00:00, 14.29it/s, avg_epoch_loss=29.2]
INFO:root:Epoch[77] Elapsed time 3.506 seconds
INFO:root:Epoch[77] Evaluation metric 'epoch_loss'=29.242862
INFO:root:Epoch[78] Learning rate is 0.0005
100%|████████

{'Coverage[0.1]': 0.07992286265266761,
 'Coverage[0.2]': 0.18127276623098368,
 'Coverage[0.3]': 0.2644096850224988,
 'Coverage[0.4]': 0.34218984358260185,
 'Coverage[0.5]': 0.42275551746303813,
 'Coverage[0.6]': 0.48746518105849607,
 'Coverage[0.7]': 0.5558174416113137,
 'Coverage[0.8]': 0.6312406256695952,
 'Coverage[0.9]': 0.7317334476108855,
 'MAE_Coverage': 0.08924362545532438,
 'MASE': 2.7788243158767516,
 'MSE': nan,
 'MSIS': 40.29709853156526,
 'ND': 0.0647189829211923,
 'NRMSE': nan,
 'QuantileLoss[0.1]': 776542.7450820922,
 'QuantileLoss[0.2]': 1098116.7230346682,
 'QuantileLoss[0.3]': 1346096.1500152587,
 'QuantileLoss[0.4]': 1508836.9166259766,
 'QuantileLoss[0.5]': 1657899.029045105,
 'QuantileLoss[0.6]': 1768447.7624084475,
 'QuantileLoss[0.7]': 1836003.5115310666,
 'QuantileLoss[0.8]': 1811673.8216278078,
 'QuantileLoss[0.9]': 1568885.9419387816,
 'RMSE': nan,
 'abs_error': 1657899.0342025757,
 'abs_target_mean': 5488.941397566194,
 'abs_target_sum': 25616889.502441406,
 

In [6]:
df[["MASE", "sMAPE", "dataset", "estimator"]]

,MASE,sMAPE,dataset,estimator
0,1.567236,0.167036,m4_hourly,SimpleFeedForwardEstimator
1,1.607215,0.134565,m4_hourly,DeepAREstimator
2,3.515622,0.193590,m4_hourly,MQCNNEstimator
3,3.272613,0.030849,m4_daily,SimpleFeedForwardEstimator
4,3.545203,0.033253,m4_daily,DeepAREstimator
5,3.806577,0.034371,m4_daily,MQCNNEstimator
6,2.821727,0.093951,m4_weekly,SimpleFeedForwardEstimator
7,3.260889,0.090204,m4_weekly,DeepAREstimator
8,2.778824,0.095051,m4_weekly,MQCNNEstimator
